In [ ]:
import psycopg2
import time

# establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    dbname = "dw_cs", 
    user = "postgres", 
    host= 'localhost',
    password = "Mu34zi72",
    port = 5432
)

In [ ]:
# function to explain-analyze a query

def explain_analyze(query, analyze = True):
    conn.rollback()
    with conn.cursor() as cur:
        if analyze:
            cur.execute(f"EXPLAIN ANALYZE {query}")
        else:
            cur.execute(f"EXPLAIN {query}")
        explain = cur.fetchall()

        for line in explain:
            print(line[0])

In [ ]:
query_materialized = """
CREATE MATERIALIZED VIEW part_lineitem AS
SELECT part.p_partkey, part.p_brand, part.p_container, lineitem.l_quantity, lineitem.l_extendedprice
FROM part JOIN lineitem ON p_partkey = l_partkey;
"""

explain_analyze(query_materialized)

In [ ]:
conn.rollback()
with conn.cursor() as cur:
    
    cur.execute("SELECT pg_total_relation_size('part_lineitem');")
    size = cur.fetchall()
    print(f"Size of materialised view: {size[0][0]/(1024**2)} MB")

In [ ]:
query_17_1 = """

CREATE VIEW average_quantity AS
SELECT
    p_partkey,
    AVG(l_quantity) AS avg_quantity
FROM
    part_lineitem
GROUP BY
    p_partkey;

SELECT
    p_partkey,
    SUM(l_extendedprice) / 7.0 AS avg_yearly
FROM
    part_lineitem
WHERE
    p_brand = 'Brand#23'
    AND p_container = 'MED BOX'
    AND l_quantity < (
        SELECT
            0.2 * avg_quantity
        FROM
            average_quantity
        WHERE 
            average_quantity.p_partkey = part_lineitem.p_partkey
    )
GROUP BY
    p_partkey;
"""